In [ ]:
import os
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt

# PenSimEnv

In [ ]:
from pensimpy.examples.recipe import Recipe, RecipeCombo
from pensimpy.data.constants import FS, FOIL, FG, PRES, DISCHARGE, WATER, PAA
from pensimpy.data.constants import FS_DEFAULT_PROFILE, FOIL_DEFAULT_PROFILE, FG_DEFAULT_PROFILE, \
    PRESS_DEFAULT_PROFILE, DISCHARGE_DEFAULT_PROFILE, WATER_DEFAULT_PROFILE, PAA_DEFAULT_PROFILE
from smpl.envs.pensimenv import PenSimEnvGym, PeniControlData, NUM_STEPS

In [ ]:
# set up the default recipe
recipe_dict = {FS: Recipe(FS_DEFAULT_PROFILE, FS),
            FOIL: Recipe(FOIL_DEFAULT_PROFILE, FOIL),
            FG: Recipe(FG_DEFAULT_PROFILE, FG),
            PRES: Recipe(PRESS_DEFAULT_PROFILE, PRES),
            DISCHARGE: Recipe(DISCHARGE_DEFAULT_PROFILE, DISCHARGE),
            WATER: Recipe(WATER_DEFAULT_PROFILE, WATER),
            PAA: Recipe(PAA_DEFAULT_PROFILE, PAA)}

recipe_combo = RecipeCombo(recipe_dict=recipe_dict)
# set up the environment
normalize=False
env = PenSimEnvGym(recipe_combo=recipe_combo, normalize=normalize)
state = env.reset(normalize=normalize, random_seed_ref=6886)
# load one batch of the sampled data
load_just_a_file='smpl/datasets/pensimenv/random_batch_0.csv'
dataset_obj = PeniControlData(load_just_a_file=load_just_a_file, normalize=normalize)
if dataset_obj.file_list:
    print('Penicillin_Control_Challenge data correctly initialized.')
else:
    raise ValueError("Penicillin_Control_Challenge data initialization failed.")
dataset = dataset_obj.get_dataset()

total_reward = 0.0
for step in range(NUM_STEPS):
    state, reward, done, info = env.step(dataset['actions'][step].tolist())
    total_reward += reward
    if step%1000 == 0:
        print("reward, total_reward:",reward, total_reward)
print("your total reward is (by default, should be around 3224):", total_reward)

# BeerFMTEnv

In [ ]:
from smpl.envs.beerfmtenv import BeerFMTEnvGym, MAX_STEPS
from smpl.envs.utils import normalize_spaces
# initialize the default policy
profile_cons = [13] * 200 # constant actions
profile_industrial = [11 + 1 / 8 * i for i in range(25)] \
                          + [14] * 95 \
                          + [14 + 2 / 25 * i for i in range(25)] \
                          + [16] * 25 + [16 - 8 / 15 * i for i in range(14)] + [9]*16 # a simple policy

In [ ]:
profile_inuse = profile_cons
env = BeerFMTEnvGym(normalize=False)
state = env.reset(initial_state=[0, 2, 2, 130, 0, 0, 0, 0])
total_reward = 0.0
for step in tqdm(range(MAX_STEPS)):
    action = np.array([profile_inuse[step]], dtype=np.float32)
    state, reward, done, info = env.step(action)
    total_reward += reward
    if done:
        break
res_cons = info["res_forplot"]
print("your end at step", step, "and total reward is:", total_reward)
#your end at step 188 and total reward is: 12.0

In [ ]:
profile_inuse = profile_industrial
env = BeerFMTEnvGym(normalize=False)
state = env.reset(initial_state=[0, 2, 2, 130, 0, 0, 0, 0])
total_reward = 0.0
for step in tqdm(range(MAX_STEPS)):
    action = np.array([profile_inuse[step]], dtype=np.float32)
    state, reward, done, info = env.step(action)
    total_reward += reward
    if done:
        break
res_industrial = info["res_forplot"]
print("your end at step", step, "and total reward is:", total_reward)
#your end at step 145 and total reward is: 55.0

In [ ]:
# plots
plt.figure(dpi=1200)
plt.subplot(2, 2, 1)
plt.plot(profile_industrial, label='Industrial', color='blue')
plt.plot(profile_cons, label='Isothermal', color='blue', linestyle='dashed')
plt.autoscale(enable=True, axis='both', tight=True)
plt.ylim((0, 18))
plt.xlabel('Time [h]')
plt.ylabel('Temperature [\u00B0C]')
plt.title("Fermentation Profile")
plt.legend()

plt.subplot(2, 2, 2)
plt.plot(res_industrial[:, 3], label='Sugar', color='m')
plt.plot(res_cons[:, 3], color='m', linestyle='dashed')
plt.plot(res_industrial[:, 4], label='Ethanol', color='orange')
plt.plot(res_cons[:, 4], color='orange', linestyle='dashed')
plt.autoscale(enable=True, axis='both', tight=True)
plt.ylim((0, 140))
plt.xlabel('Time [h]')
plt.ylabel('Concentration [g/L]')
plt.title("Sugar and Ethanol Production")
plt.legend()

plt.subplot(2, 2, 3)
plt.plot(res_industrial[:, 0], label='Active', color='green')
plt.plot(res_cons[:, 0], color='green', linestyle='dashed')
plt.plot(res_industrial[:, 1], label='Lag', color='c')
plt.plot(res_cons[:, 1], color='c', linestyle='dashed')
plt.plot(res_industrial[:, 2], label='Dead', color='red')
plt.plot(res_cons[:, 2], color='red', linestyle='dashed')
plt.plot(res_industrial[:, 0] + res_industrial[:, 1] + res_industrial[:, 2], label='Total', color='black')
plt.plot(res_cons[:, 0] + res_cons[:, 1] + res_cons[:, 2], color='black', linestyle='dashed')
plt.autoscale(enable=True, axis='both', tight=True)
plt.ylim((0, 9))
plt.xlabel('Time [h]')
plt.ylabel('Concentration [g/L]')
plt.title("Biomass Evolution")
plt.legend()

plt.subplot(2, 2, 4)
plt.plot(res_industrial[:, 5], label='Diacetyl', color='darkgoldenrod')
plt.plot(res_cons[:, 5], color='darkgoldenrod', linestyle='dashed')
plt.plot(res_industrial[:, 6], label='Ethyl Acelate', color='grey')
plt.plot(res_cons[:, 6], color='grey', linestyle='dashed')
plt.autoscale(enable=True, axis='both', tight=True)
plt.ylim((0, 1.6))
plt.xlabel('Time [h]')
plt.ylabel('Concentration [ppm]')
plt.title("By-Product Production")
plt.legend()


plt.tight_layout()
plt.show()

# AtropineEnv

In [2]:
import numpy as np
from smpl.envs.atropineenv import AtropineEnvGym
env = AtropineEnvGym(normalize=False)
init_state = env.sample_initial_state(no_sample=True)
env.reset(initial_state=init_state)
print("init_state.shape", init_state.shape)
state, reward, done, info = env.step(np.array([-1.03856889e-06, -8.12998852e-05, -1.66432480e-05,  1.81665393e-05]))
print("state.shape", init_state.shape)
print("reward", reward) # should be -1.7763568394002505e-15

OSError: Failed to interpret file <_io.BytesIO object at 0x7f5dd2dfef90> as a pickle

In [ ]:
# or, if we want to run a full loop to see the plot:
env = AtropineEnvGym(reward_on_steady=False, reward_on_absolute_efactor=False, observation_include_t=True, observation_include_action=True, observation_include_uss=True, observation_include_ess=True, observation_include_e=True, observation_include_kf=True, observation_include_z=True, observation_include_x=True)
init_state = env.sample_initial_state(no_sample=True)
init_state = env.reset(initial_state=init_state)
for i in range(60):
    state, reward, done, info = env.step(np.ones(4)*0.5)
env.plot()

In [ ]:
# or, use the provided MPC controller:
import numpy as np
from smpl.envs.atropineenv import AtropineEnvGym
from smpl.envs.atropineenv import AtropineMPC
mpc = AtropineMPC()
env = AtropineEnvGym(normalize=False)
init_state = env.sample_initial_state(no_sample=True)
init_state = env.reset(initial_state=init_state)
state = init_state
for i in range(60):
    action = mpc.predict(state)
    state, reward, done, info = env.step(action)
env.plot()

# ReactorEnv

In [1]:
from smpl.envs.reactorenv import ReactorEnvGym
env = ReactorEnvGym(normalize=False)
init_state = env.reset(initial_state=[0.8778252, 51.34660837, 0.659]) # init with steady state (setpoint)
state, reward, done, info = env.step([26.85, 0.1]) # feed in the steady action (setpoint action)
print("state", state) # should be [0.8778252, 51.34660837, 0.659], we still get the same steady state

state [ 0.8778252  51.34660727  0.65899998]


# MAbEnv

In [ ]:
from smpl.envs.mabenv import MAbEnvGym, xscale, uscale
import numpy as np
env = MAbEnvGym(normalize=False, standard_reward_style='setpoint')
xss = env.xss / xscale
uss = env.uss / uscale
env.reset(xss)
observation, reward, done, info = env.step(uss, normalize=False)
assert (np.square((observation - xss)[:19])).sum() < 1e-10 # it is steady, so given steady action at steady state, we should still get steady state
env = MAbEnvGym(normalize=False, standard_reward_style='productivity')
env.reset(xss)
observation, reward, done, info = env.step(uss, normalize=False)
print(reward) # should be around 1349.4986